First install the following packages:

- `requests`
- `tqdm`
- `xarray-leaflet`
- `dask`

The recommended way is: `conda install -c conda-forge requests tqdm xarray_leaflet dask`.

We start by importing the modules that we will need:

In [1]:
import requests
import os
import warnings
from tqdm import tqdm
import numpy as np
import zipfile
import rioxarray
import xarray_leaflet
from ipyleaflet import Map, basemaps, LayersControl, WidgetControl
from ipywidgets import FloatSlider
import matplotlib.pyplot as plt

We will display the DEM (digital elevation model) for Asia from the [HydroSHEDS](https://hydrosheds.org) dataset, which represents the terrain. Let's first download the data:

In [3]:
url = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/hydrosheds/sa_30s_zip_grid/as_dem_30s_grid.zip'
filename = os.path.basename(url)
name = filename[:filename.find('_grid')]
adffile = os.path.join(name, name, 'w001001.adf')

if not os.path.exists(adffile):
    r = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        for chunk in tqdm(r.iter_content(chunk_size=1024), total=(total_length/1024) + 1):
            if chunk:
                f.write(chunk)
                f.flush()
    zip = zipfile.ZipFile(filename)
    zip.extractall('.')

It is a dataset that [Rasterio](https://rasterio.readthedocs.io) can open, but in order to get a nice `DataArray` with all the metadata taken into account, we open it with rioxarray:

In [4]:
da = rioxarray.open_rasterio(adffile, masked=True)
da

<xarray.DataArray (band: 1, y: 8400, x: 15000)>
[126000000 values with dtype=float32]
Coordinates:
  * band         (band) int32 1
  * x            (x) float64 55.0 55.01 55.02 55.03 ... 180.0 180.0 180.0 180.0
  * y            (y) float64 60.0 59.99 59.98 59.97 ... -9.979 -9.988 -9.996
    spatial_ref  int32 0
Attributes:
    scale_factor:  1.0
    add_offset:    0.0

The projection is `EPSG:4326` (aka `WGS84`). Here the coordinate `x` corresponds to longitudes, and `y` to latitudes (in degrees). There is only one band.

In [5]:
da = da.sel(band=1)
da.name = 'DEM'

The dataset can be too big to hold in memory, so we will chunk it into smaller pieces. That will also improve performances as the generation of a tile can be done in parallel using Dask.

In [5]:
da = da.chunk((1000, 1000))
warnings.filterwarnings("ignore")

We just need to create a map before passing it to our `DataArray` extension.

In [6]:
m = Map(center=[40, 115], zoom=3, basemap=basemaps.CartoDB.DarkMatter, interpolation='nearest')
m

Map(center=[40, 115], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

To show our data on the map, we call `leaflet.plot()` on our `DataArray`, and pass the map as parameter. We get back a layer, that we can further control with e.g. a slider to set the opacity.

In [7]:
l = da.leaflet.plot(m, colormap=plt.cm.terrain)
# l.interact(opacity=(0., 1.))

Url()

We could have a slider outside of the map, but we are going to insert it in the map. We will also insert a layer control.

In [8]:
layers_control = LayersControl(position='topright')
m.add_control(layers_control)

opacity_slider = FloatSlider(description='Opacity:', min=0, max=1, value=1)

def set_opacity(change):
    l.opacity = change['new']

opacity_slider.observe(set_opacity, names='value')
slider_control = WidgetControl(widget=opacity_slider, position='bottomleft')
m.add_control(slider_control)

The `select()` method allows to select a region by clicking and dragging a box on the map (first click on the █ button).

In [9]:
da.leaflet.select()

You can then get the selected data array back, and e.g. plot it.

In [ ]:
da_selected = da.leaflet.get_selection()

In [ ]:
if da_selected is not None:
    da_selected.plot.imshow()

In [ ]:
da.leaflet.unselect()  # remove the selection buttons